### Imports

In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
def show_image(title, image):
    image = cv.resize(image, (0, 0), fx=0.2, fy=0.2)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Functia de detectie a conturului

### Functia de modificare a imaginii initiale cu operatii morfologice

In [3]:
### fereastra de proprietati custom
def empty(a):
    pass


def createGUI():
    cv.namedWindow("trackBars", cv.WINDOW_NORMAL)
    # cv.resizeWindow("trackBarks", 640, 240)
    cv.createTrackbar("Hue Min", "trackBars", 0, 179, empty)
    cv.createTrackbar("Hue Max", "trackBars", 179, 179, empty)
    cv.createTrackbar("Sat Min", "trackBars", 100, 255, empty)
    cv.createTrackbar("Sat Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("Val Min", "trackBars", 100, 255, empty)
    cv.createTrackbar("Val Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("MedBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("GausBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("Sharp alp", "trackBars", 12, 20, empty)
    cv.createTrackbar("Sharp bet", "trackBars", 8, 20, empty)
    cv.createTrackbar("Sharp gam", "trackBars", 0, 20, empty)
    cv.createTrackbar("tresmin", "trackBars", 0, 255, empty)
    cv.createTrackbar("tresmax", "trackBars", 255, 255, empty)
    cv.createTrackbar("cannymin", "trackBars", 0, 500, empty)
    cv.createTrackbar("cannymax", "trackBars", 0, 500, empty)


In [4]:
def imgModify(imgOriginala):
    h, w, _ = imgOriginala.shape
    # print(h, w)
    img = cv.resize(imgOriginala, (0, 0), fx=0.2, fy=0.2)

    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    createGUI()
    while True:
        h_min = cv.getTrackbarPos("Hue Min", "trackBars")
        h_max = cv.getTrackbarPos("Hue Max", "trackBars")
        s_min = cv.getTrackbarPos("Sat Min", "trackBars")
        s_max = cv.getTrackbarPos("Sat Max", "trackBars")
        v_min = cv.getTrackbarPos("Val Min", "trackBars")
        v_max = cv.getTrackbarPos("Val Max", "trackBars")
        median = cv.getTrackbarPos("MedBlur", "trackBars")
        gaussian = cv.getTrackbarPos("GausBlur", "trackBars")
        alpha = cv.getTrackbarPos("Sharp alp", "trackBars")
        beta = cv.getTrackbarPos("Sharp bet", "trackBars")
        gama = cv.getTrackbarPos("Sharp gam", "trackBars")
        canny_min = cv.getTrackbarPos("cannymin", "trackBars")
        canny_max = cv.getTrackbarPos("cannymax", "trackBars")
        treshold_min = cv.getTrackbarPos("tresmin", "trackBars")
        treshold_max = cv.getTrackbarPos("tresmax", "trackBars")

        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        mask = cv.inRange(img_hsv, lower, upper)

        img_result = cv.bitwise_and(img, img, mask=mask)

        ###HSV to GRAY
        h, s, v = cv.split(img_result)
        img_result = v

        img_m_blur = cv.medianBlur(img_result, median)
        img_g_blur = cv.GaussianBlur(img_result, (7, 7), gaussian)
        # img_sharpened = cv.addWeighted(img_m_blur, 1.2, img_g_blur, -0.8, gama)
        img_sharpened = cv.addWeighted(img_m_blur, alpha / 10, img_g_blur, (-1) * beta / 10, gama)
        _, thresh = cv.threshold(img_sharpened, treshold_min, treshold_max, cv.THRESH_BINARY)

        edges = cv.Canny(thresh, canny_min, canny_max)

        contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        img_final = img.copy()
        print(len(contours))
        max_area = 0
        x, y, w, h = 0,0,0,0
        for cont in contours:
            area = cv.contourArea(cont)
            if area > max_area:
                max_area = area
                peri = cv.arcLength(cont, True)
                cv.drawContours(img_final, cont, -1, (255, 0, 0), 3)
                aprox = cv.approxPolyDP(cont, 0.02 * peri, True)
                x, y, w, h = cv.boundingRect(aprox)
        cv.rectangle(img_final, (x, y), (x + w, y + h), (255, 255, 0), 2)


        cv.imshow("Thresh", thresh)
        cv.imshow("Edges", edges)
        cv.imshow("Original", img)
        cv.imshow("BlurG", img_g_blur)
        cv.imshow("BlurM", img_m_blur)
        cv.imshow("sharpened", img_sharpened)
        cv.imshow("Result", img_result)
        cv.imshow("AFISARE PATRAT", img_final)

        if cv.waitKey(0) & 0xFF == ord('p'):
            break

    cv.destroyAllWindows()

In [ ]:
path = 'antrenare/'
img = cv.imread(path + "/1_01.jpg")

# imgModify(img)
files = os.listdir(path)
print(files)
for file in files:
    if file[len(file) - 3:len(file)] == "jpg":
        img = cv.imread(path + file)
        imgModify(img)
# show_image('test', img)

['1_01.jpg', '1_01.txt', '1_02.jpg', '1_02.txt', '1_03.jpg', '1_03.txt', '1_04.jpg', '1_04.txt', '1_05.jpg', '1_05.txt', '1_06.jpg', '1_06.txt', '1_07.jpg', '1_07.txt', '1_08.jpg', '1_08.txt', '1_09.jpg', '1_09.txt', '1_10.jpg', '1_10.txt', '1_11.jpg', '1_11.txt', '1_12.jpg', '1_12.txt', '1_13.jpg', '1_13.txt', '1_14.jpg', '1_14.txt', '1_15.jpg', '1_15.txt', '1_16.jpg', '1_16.txt', '1_17.jpg', '1_17.txt', '1_18.jpg', '1_18.txt', '1_19.jpg', '1_19.txt', '1_20.jpg', '1_20.txt', '2_01.jpg', '2_01.txt', '2_02.jpg', '2_02.txt', '2_03.jpg', '2_03.txt', '2_04.jpg', '2_04.txt', '2_05.jpg', '2_05.txt', '2_06.jpg', '2_06.txt', '2_07.jpg', '2_07.txt', '2_08.jpg', '2_08.txt', '2_09.jpg', '2_09.txt', '2_10.jpg', '2_10.txt', '2_11.jpg', '2_11.txt', '2_12.jpg', '2_12.txt', '2_13.jpg', '2_13.txt', '2_14.jpg', '2_14.txt', '2_15.jpg', '2_15.txt', '2_16.jpg', '2_16.txt', '2_17.jpg', '2_17.txt', '2_18.jpg', '2_18.txt', '2_19.jpg', '2_19.txt', '2_20.jpg', '2_20.txt', '3_01.jpg', '3_01.txt', '3_02.jpg', '3_